# Heart Disease Prediction Model - Exploratory Analysis #
## Author: Madison Little ##
## Date: Sep. 4, 2024 ##

This initial report explores the data's quality, completeness, and distribution.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from charts import categorical_ct
from charts import grouped_categorical_ct

The following two defined functions are used throughout the report to create similar charts for each of the categorical values.
I defined them in this notebook so I would not have to read the categorical_mapping.json file each time the function is called or pass the json data as a parameter to the function.  This way, the functions can access the mapping as a global variable.

In [ ]:
df = pd.read_csv('data/heart_disease_prediction.csv')
df.head()

In [ ]:
print(f"Features: {len(df.columns)}")
print(f"Observations: {len(df)}")

In [ ]:
df.describe(include='all')

## Observations ##
Patient ages range from 28 to 77, with a mean of about 53 years.

It appears that RestingBP and Cholesterol have missing values, as their minimums are 0.0--a value that does not make sense for those measurements.

MaxHR is more ambiguous.  There is a minimum value of 60BPM, but the lower quartile is 120BPM. This may mean that 60 is a placeholder value for unrecorded heartrate. If these heartrates were achieved during exercise, it is unlikley that 60BPM was a recorded value.

The cholesterol column has a max value of 603, which is 2.5 times larger than what is considered "high" cholesterol.  Based on the IQR, this is an extreme outlier.  According to the American Heart Association, such a cholesterol level is caused by a rare genetic disease called familial hypercholesterolemia.


In [ ]:
fig = categorical_ct(df, 'Sex')
fig.update_layout(
    title="Sex of Patients",
    xaxis_title="Sex",
    yaxis_title="Count of Patients")


There are more male patients represented in the data than female patients (~80% men)

In [ ]:
fig = categorical_ct(df, 'ChestPainType')
fig.update_layout(
    title="Type of Patient Chest Pain",
    xaxis_title="Chest Pain Type",
    yaxis_title="Count of Patients")

Asymptomatic chest pain (meaning no chest pain) is the most common in the data.

In [ ]:
fig = categorical_ct(df, 'HeartDisease')
fig.update_layout(
    title="Heart Disease Cases",
    xaxis_title="Test Result",
    yaxis_title="Count of Patients")

The dataset is pretty evenly split between patients with heart disease and those without heart disease. This means we have ample data for both classes, and we should be able to classify a test patient into one of the two categories. 

In [ ]:
fig = categorical_ct(df, 'FastingBS')
fig.update_layout(
    title="Patient Blood Sugar Levels",
    xaxis_title="Blod Sugar Range",
    yaxis_title="Count of Patients")

The data is already split into categorical blood sugar ranges--above or below 120mg/dl.  A quick search revealed that the normal range is under 120mg/dl.  Most patients in the data are therefore non-diabetic.

In [ ]:

fig = categorical_ct(df, 'RestingECG')
fig.update_layout(
    title='Patient Resting Electrocardiogram Results',
    xaxis_title="Test Result",
    yaxis_title="Count of Patients")

Most patients have a "normal" ECG test result. There are two categories that are therefore "abnormal".  If we combine the two abnormal groups, then we have a much closer split between normal and abnormal ECG readings.

In [ ]:

fig = categorical_ct(df, 'ExerciseAngina')
fig.update_layout(
    title='Presense of Exercise-Induced Angina',
    xaxis_title="Angina Present?",
    yaxis_title="Count of Patients")


Most patients do not experience angina after exercise, but the split is very close.  It is unclear how this metric relates to the "ChestPainType" data, where 

In [ ]:
fig = categorical_ct(df, 'ST_Slope')
fig.update_layout(
    title='Patients\' ECG ST-Segment Slope',
    xaxis_title="Test Result",
    yaxis_title="Count of Patients")

In [ ]:
fig = grouped_categorical_ct(df, grouper='Sex', counter='HeartDisease', colors=['red', 'green'])
fig.update_layout(
    title='Heart Disease in Patients by Sex',
    xaxis_title="Sex",
    yaxis_title="Count of Patients",
    legend_title="Heart Disease Result")


In [ ]:
fig = grouped_categorical_ct(df, grouper='ChestPainType', counter='HeartDisease', colors=['red', 'green'])
fig.update_layout(
    title='Presense of Heart Disease By Chest Pain Type',
    xaxis_title="Chest Pain Type",
    yaxis_title="Count of Patients",
    legend_title="Heart Disease Result")

77% of all heart disease cases had no chest pain symptoms. Strangely, more healthy patients reported chest pain than patients with heart disease.  Non-anginal pain was the most common type of chest pain reported by patients with heart disease.  The most common pain reported by healthy patients was atypical angina. Typical angina was the least reported chest pain type overall, with almost equal numbers of healthy and heart disease patients.

In [ ]:
fig = grouped_categorical_ct(df, grouper='FastingBS', counter='HeartDisease', colors=['red', 'green'])
fig.update_layout(
    title='Heart Disease in Patients by Blood Sugar Level',
    xaxis_title="Blood Sugar Range",
    yaxis_title="Count of Patients",
    legend_title="Heart Disease Result")

Non-diabetic patients are more common than diabetic patients overall.  Among non-diabetic patients, there are about the same number of patients with and without heart disease.  However, there is much larger portion of diabetic patients that also have heart disease.  While the absense of diabetes does not imply the absense of heart disease, the presense of diabetes is a strong indicator of heart disease.

In [ ]:
fig = grouped_categorical_ct(df, grouper='RestingECG', counter='HeartDisease', colors=['red', 'green'])
fig.update_layout(
    title='Heart Disease in Patients by ECG Result',
    xaxis_title="ECG Reading",
    yaxis_title="Count of Patients",
    legend_title="Heart Disease Result")

There are fairly similar distributions of healthy and heart disease patients across the ECG reading categories.  The ST-T wave abnormality group has the most drastic difference in frequency.  It is more common for a person with heart disease to display an ST-T wave abnormality than a healthy patient, which could indicate another symptom or risk factor for heart disease.

In [ ]:
fig = grouped_categorical_ct(df, grouper='ExerciseAngina', counter='HeartDisease', colors=['red', 'green'])
fig.update_layout(
    title='Exercise-Induced Angina and Heart Disease',
    xaxis_title="Angina Induced by Exercise?",
    yaxis_title="Count of Patients",
    legend_title="Heart Disease Result")

Most patients that have heart disease experience exercised-induced angina, and very few healthy patients display this symptom.  This may be a good feature to use in our predictive model. It is important to note that there is still a significant portion of patients with heart disease that do not experience angina after exercising (false negative risk).

In [ ]:
fig = grouped_categorical_ct(df, grouper='ST_Slope', counter='HeartDisease', colors=['red', 'green'])
fig.update_layout(
    title='Heart Disease in Patients by ST-Segment Slope',
    xaxis_title="ST-Segment Slope",
    yaxis_title="Count of Patients",
    legend_title="Heart Disease Result")

After researching this measurement, I found that an upward slope of the ST-segment of an ECG reading is considered normal.  Downward or horizontal (flat) slopes are considered ST-segment depressions.

A large majority of those with heart disease display a flat ST-segment slope, while most healthy patients display a normal upward slope.  There are fewer patients overall that display a downward slope, but this is more common among those with heart disease rather than healthy individuals. Some healthy patients display a flat ST-segment slope. This could be an indicator that the patient is at risk of heart disease given that the upslope is more common among healthy individuals. More grouping analysis is needed for this metric, but it could make another strong feature for my model.

In [ ]:
# Cleaning the values that don't make sense: RestingBP and Cholesterol
df[df['RestingBP']==0]

Only one patient has a recorded resting BP of 0, so we will remove that record.

In [ ]:
df = df[df['RestingBP'] != 0]

In [ ]:
df[df['Cholesterol']==0]

There are significantly more patients without a cholesterol reading.  We will fill these values with the median of the patient's sex, age group, and heart disease status.

In [ ]:
age_bins = list(range(0, 100, 5))
df['AgeRange'] = pd.cut(df['Age'], bins=age_bins)
df_reduced = df[df['Cholesterol']!=0]
medians = df_reduced.groupby(['Sex', 'AgeRange', 'HeartDisease'], observed=True).agg(MedCholesterol=('Cholesterol', 'median'))
df = df.merge(medians, how='left', on=['Sex', 'AgeRange', 'HeartDisease'])
df['Cholesterol'] = np.where(df['Cholesterol'] == 0, df['MedCholesterol'], df['Cholesterol'])
df.drop(columns=['MedCholesterol'])
